#### Install the required libraries 
</br>

In [1]:
!pip install huggingface_hub
!pip install langchain
!pip install langchain_community
!pip install pymupdf
!pip install faiss-cpu


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip



#### Import all the required packages

In [2]:
from huggingface_hub import login
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders.pdf import PyMuPDFLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain_community.llms import Replicate
from langchain_core.prompts import PromptTemplate


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Document Loading 
#Load the directory that has domain specific documents
dir_loader = DirectoryLoader('Dataset/', 
                             glob="**/*.pdf", 
                             show_progress=True, use_multithreading=True,
                             loader_cls=PyMuPDFLoader)

data = dir_loader.load()

#Document Transformation
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
docs = text_splitter.split_documents(data)

print(len(docs))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 10.59it/s]

106


In [4]:
#Check the splitted document sample
print(docs[0].metadata)
print(docs[0].page_content)

{'source': 'Dataset/Evaluating AI for Law.pdf', 'file_path': 'Dataset/Evaluating AI for Law.pdf', 'page': 0, 'total_pages': 16, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'pdfTeX-1.40.25', 'creationDate': 'D:20240419005128Z', 'modDate': 'D:20240419005128Z', 'trapped': ''}
Evaluating AI for Law: Bridging the Gap with
Open-Source Solutions
Rohan Bhambhoria1,2[0000−0002−2597−670X],
Samuel Dahan1,2,3[0000−0002−1079−8998],
Jonathan Li2[0000−0002−7095−805X], and
Xiaodan Zhu1,2[0000−0003−3856−3696]
1 Queen’s University, Kingston ON K7L3N6, Canada
{r.bhambhoria, samuel.dahan, jxl, xiaodan.zhu}@queensu.ca
2 Ingenuity Labs
3 Cornell University
Abstract. This study evaluates the performance of general-purpose AI,
like ChatGPT, in legal question-answering tasks, highlighting significant
risks to legal professionals and clients. It suggests leveraging founda-
tional models enhanced by domain-specific knowledge to over

In [6]:
#Using HuggingFace's BGE Embedding model 
huggingface_embeddings = HuggingFaceBgeEmbeddings(
                        	model_name="BAAI/bge-small-en-v1.5",  # alternatively use "sentence-transformers/all-MiniLM-l6-v2" 
                                                                  # for a light and faster experience.
                        	model_kwargs={'device': 'cpu'},
                        	encode_kwargs={'normalize_embeddings': True} 
                        )

In [7]:
text = "Testing"
query_result = huggingface_embeddings.embed_query(text)
query_result

[-0.04916955903172493,
 0.0363210029900074,
 0.010675057768821716,
 0.020139532163739204,
 -0.02208401821553707,
 0.009389788843691349,
 0.03015921451151371,
 0.05626610293984413,
 -0.0009100144379772246,
 -0.03351806849241257,
 -0.016063209623098373,
 -0.024940671399235725,
 0.016899123787879944,
 0.030053798109292984,
 0.005090599413961172,
 -0.007806930225342512,
 -0.0032484543044120073,
 0.002750767394900322,
 -0.023655923083424568,
 0.028179416432976723,
 0.027786096557974815,
 0.03960951045155525,
 0.012235529720783234,
 -0.01770024187862873,
 0.0031805322505533695,
 0.041944511234760284,
 0.05696551501750946,
 -0.05506322160363197,
 -0.03417317941784859,
 -0.12369446456432343,
 -0.016410015523433685,
 -0.02873205952346325,
 0.019709547981619835,
 0.016292748972773552,
 0.017873598262667656,
 -0.0013450781116262078,
 0.0017727789236232638,
 0.013763399794697762,
 0.011978093534708023,
 -0.002964153653010726,
 0.039542991667985916,
 -0.004725602455437183,
 -0.038533128798007965,
 

In [8]:
# Create a FAISS Vector Store from the documents and the embeddings from embedding model
db = FAISS.from_documents(docs, huggingface_embeddings)

# Test the retrieval of similar documents with a sample question along with the similarity scores
question = "What is the conclusion of the paper 'Evaluating AI for Law: Bridging the Gap with Open-Source Solutions' ?"
searchDocs = db.similarity_search_with_relevance_scores(question)

for doc in searchDocs[:3]:
    print("*"*25)
    print(doc[0].metadata)
    print("\n Similarity Score: "+ str(doc[1]))
    print("\n"+doc[0].page_content)

*************************
{'source': 'Dataset/Evaluating AI for Law.pdf', 'file_path': 'Dataset/Evaluating AI for Law.pdf', 'page': 0, 'total_pages': 16, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'pdfTeX-1.40.25', 'creationDate': 'D:20240419005128Z', 'modDate': 'D:20240419005128Z', 'trapped': ''}

 Similarity Score: 0.8130470113506805

Evaluating AI for Law: Bridging the Gap with
Open-Source Solutions
Rohan Bhambhoria1,2[0000−0002−2597−670X],
Samuel Dahan1,2,3[0000−0002−1079−8998],
Jonathan Li2[0000−0002−7095−805X], and
Xiaodan Zhu1,2[0000−0003−3856−3696]
1 Queen’s University, Kingston ON K7L3N6, Canada
{r.bhambhoria, samuel.dahan, jxl, xiaodan.zhu}@queensu.ca
2 Ingenuity Labs
3 Cornell University
Abstract. This study evaluates the performance of general-purpose AI,
like ChatGPT, in legal question-answering tasks, highlighting significant
risks to legal professionals and clients. It suggests leveraging f

In [9]:
retriever = db.as_retriever() 
docs = retriever.get_relevant_documents(question)
docs

[Document(page_content='Evaluating AI for Law: Bridging the Gap with\nOpen-Source Solutions\nRohan Bhambhoria1,2[0000−0002−2597−670X],\nSamuel Dahan1,2,3[0000−0002−1079−8998],\nJonathan Li2[0000−0002−7095−805X], and\nXiaodan Zhu1,2[0000−0003−3856−3696]\n1 Queen’s University, Kingston ON K7L3N6, Canada\n{r.bhambhoria, samuel.dahan, jxl, xiaodan.zhu}@queensu.ca\n2 Ingenuity Labs\n3 Cornell University\nAbstract. This study evaluates the performance of general-purpose AI,\nlike ChatGPT, in legal question-answering tasks, highlighting significant\nrisks to legal professionals and clients. It suggests leveraging founda-\ntional models enhanced by domain-specific knowledge to overcome these\nissues. The paper advocates for creating open-source legal AI systems\nto improve accuracy, transparency, and narrative diversity, addressing\ngeneral AI’s shortcomings in legal contexts.\nKeywords: Law · Open-Source · Large Language Models.\n1\nIntroduction\nIn recent times, the rise of Large Language Mo

#### Provide API token to access replicate models

In [22]:
from dotenv import dotenv_values

config = dotenv_values("keys.env") 
os.environ["REPLICATE_API_TOKEN"] = config['replicate_api_token'] 

In [23]:
# Replicate runs machine learning models in the cloud

llm = Replicate(
                model="meta/meta-llama-3-70b-instruct", 
                model_kwargs={"temperature": 0.1, "max_length": 500, "top_p": 1},
                )

#### --------------------------------------------------------------------------------------------------------------------------------------

### Enough Code!  Let's chitchat!
#### ------------------------------------------------------------------------------------------------------------------------------------

In [29]:

print(llm.invoke("When was the paper titled 'Evaluating AI for Law: Bridging the Gap with Open-Source Solutions', with the author Rohan Bhambhoria, published?"))


I'm happy to help! However, I need more information to provide an accurate answer. After conducting a search, I couldn't find a specific paper titled "Evaluating AI for Law: Bridging the Gap with Open-Source Solutions" by Rohan Bhambhoria.

Could you please provide more context or details about the paper, such as:

1. The publication or journal it was published in?
2. The year or approximate timeframe you're looking for?
3. Any other relevant information that might help me narrow down the search?

I'll do my best to assist you in finding the answer!


In [26]:

print(llm.invoke("What is the conclusion of the paper 'Evaluating AI for Law: Bridging the Gap with Open-Source Solutions' ?"))


I'd be happy to help you with that!

The paper "Evaluating AI for Law: Bridging the Gap with Open-Source Solutions" explores the potential of artificial intelligence (AI) in the legal domain and proposes open-source solutions to address the challenges of evaluating AI systems for legal applications.

After reviewing the paper, I found that the conclusion highlights the following key points:

1. **Need for standardized evaluation frameworks**: The authors emphasize the importance of developing standardized evaluation frameworks for AI systems in law, which can ensure consistency, transparency, and reproducibility across different use cases.
2. **Open-source solutions as a bridge**: The paper concludes that open-source solutions can bridge the gap between AI research and practical legal applications by providing accessible, customizable, and community-driven tools for evaluating AI systems.
3. **Collaboration between academia, industry, and legal practitioners**: The authors stress the n

#### --------------------------------------------------------------------------------------------------------------------------------------

#### Time to employ RAG system! 
#### --------------------------------------------------------------------------------------------------------------------------------------

In [45]:
# instructions to the LLM
prompt_template = """
Use the following pieces of context to answer the question at the end. Please follow the following rules:
1. If you don't know the answer, don't try to make up an answer. Just say "Sorry I can't find the final answer!".
2. If you find the answer, write the answer in a concise way with five sentences maximum.

{context}

Question: {question}

Helpful Answer:
"""

# Vector db as a retriever with similarity search and uses 4 source documents
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4}) 

# Define the prompt template with variables
PROMPT = PromptTemplate(
	template=prompt_template, input_variables=["context", "question"]
)

# Create the retrievalQA by setting LLM, retriever and other parameters
retrievalQA = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=retriever,
	return_source_documents=True,
	chain_type_kwargs={"prompt": PROMPT}
)

def print_readable(result):
    print("\n"+"-"*100)
    print(result)
    print("\n"+"-"*100)

#### --------------------------------------------------------------------------------------------------------------------------------------

#### Shall we test our RAG System? 
#### --------------------------------------------------------------------------------------------------------------------------------------

In [46]:

result = retrievalQA.invoke({"query": "When was the paper titled 'Evaluating AI for Law: Bridging the Gap with Open-Source Solutions', with the author Rohan Bhambhoria, published?"})
print_readable(result['result'])


----------------------------------------------------------------------------------------------------
Sorry I can't find the final answer! The provided context does not mention the publication date of the paper.

----------------------------------------------------------------------------------------------------


In [47]:

result = retrievalQA.invoke({"query": "What is the conclusion of the paper 'Evaluating AI for Law: Bridging the Gap with Open-Source Solutions' ?"})
print_readable(result['result'])



----------------------------------------------------------------------------------------------------
The conclusion of the paper is that artificial intelligence has a profound impact on legal applications, and the authors advocate for open-source legal language models to address the limitations of general-purpose language models in high-stakes decision making. They propose creating a legal language model called OpenJustice and introduce a real-world dataset called LegalQA to evaluate the performance of current state-of-the-art LLMs.

----------------------------------------------------------------------------------------------------


In [48]:

relevant_docs = result['source_documents']
print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')
print("*" * 100)
for i, doc in enumerate(relevant_docs):
	print(
		f"Relevant Document #{i + 1}:\nSource file: {doc.metadata['source']}, Page: {doc.metadata['page']}\nContent: {doc.page_content}")
	print("-" * 100)


There are 4 documents retrieved which are relevant to the query.
****************************************************************************************************
Relevant Document #1:
Source file: Dataset/Evaluating AI for Law.pdf, Page: 0
Content: Evaluating AI for Law: Bridging the Gap with
Open-Source Solutions
Rohan Bhambhoria1,2[0000−0002−2597−670X],
Samuel Dahan1,2,3[0000−0002−1079−8998],
Jonathan Li2[0000−0002−7095−805X], and
Xiaodan Zhu1,2[0000−0003−3856−3696]
1 Queen’s University, Kingston ON K7L3N6, Canada
{r.bhambhoria, samuel.dahan, jxl, xiaodan.zhu}@queensu.ca
2 Ingenuity Labs
3 Cornell University
Abstract. This study evaluates the performance of general-purpose AI,
like ChatGPT, in legal question-answering tasks, highlighting significant
risks to legal professionals and clients. It suggests leveraging founda-
tional models enhanced by domain-specific knowledge to overcome these
issues. The paper advocates for creating open-source legal AI systems
to improve accuracy, 

#### --------------------------------------------------------------------------------------------------------------------------------------

In [49]:

print(llm.invoke("what is the dataset used in the paper titled Evaluating AI for Law: Bridging the Gap with Open-Source Solutions?"))

After conducting a search, I found that the paper "Evaluating AI for Law: Bridging the Gap with Open-Source Solutions" uses several datasets to evaluate AI models for legal applications. The specific datasets used in the paper are:

1. **CaseLaw**: A dataset of 7,700 court cases from the United States, annotated with relevant information such as case type, jurisdiction, and outcome.
2. **European Court of Human Rights (ECHR)**: A dataset of 11,000 court cases from the European Court of Human Rights, annotated with information such as article violations and outcome.
3. **Stanford Large Dataset (SLDT)**: A dataset of 150,000 court cases from the United States, annotated with information such as case type, jurisdiction, and outcome.
4. **Legal Information Institute (LII)**: A dataset of 100,000 court cases from the United States, annotated with information such as case type, jurisdiction, and outcome.

These datasets are used to evaluate various AI models for tasks such as case classifica

#### --------------------------------------------------------------------------------------------------------------------------------------

In [50]:

res = retrievalQA.invoke({"query": "what is the dataset used in the paper titled Evaluating AI for Law: Bridging the Gap with Open-Source Solutions?"})
print_readable(res['result'])


----------------------------------------------------------------------------------------------------
The dataset used in the paper is called LegalQA, which is a high-quality dataset manually annotated by legal experts.

----------------------------------------------------------------------------------------------------


</br></br></br></br></br></br></br></br></br></br>

### What about continuous chatting?
#### --------------------------------------------------------------------------------------------------------------------------------------

In [91]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import (ConversationalRetrievalChain)
from langchain.prompts import PromptTemplate

template = """Answer the question in your own words as truthfully as possible from the context given to you.
If you do not know the answer to the question, simply respond with "I don't know. Can you ask another question".
If questions are asked where there is no relevant context available, simply respond with "I don't know. Please ask a question relevant to the documents"
Context: {context}


{chat_history}
Human: {question}
Assistant:"""

prompt = PromptTemplate(
    input_variables=["context", "chat_history", "question"], template=template
)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer')

# Create the custom chain
chain = ConversationalRetrievalChain.from_llm(
                    llm=llm, retriever=retriever, memory=memory,
                    # get_chat_history=get_chat_history, 
                    return_source_documents=True,
                    combine_docs_chain_kwargs={'prompt': prompt}
            )
    

In [94]:

res = chain("what is the dataset used in the paper titled Evaluating AI for Law: Bridging the Gap with Open-Source Solutions?")
print_readable(res['answer'])



----------------------------------------------------------------------------------------------------
The dataset used in the paper is called LegalQA, which is a high-quality dataset manually annotated by legal experts.

----------------------------------------------------------------------------------------------------


In [95]:

res = chain("Can you provide more details about this dataset?")
print_readable(res['answer'])



----------------------------------------------------------------------------------------------------
The LegalQA dataset is a high-quality dataset of over 2000 questions asked by laypeople on real legal questions and answers vetted by legal experts. The questions are sourced from an online legal community, and law students were asked to write expert answers to these questions. The dataset is designed to reflect real-world scenarios and address various dimensions desirable for a language model in aiding laypeople with legal tasks.

----------------------------------------------------------------------------------------------------


In [96]:

res = chain("What is the conclusion of this paper?")
print_readable(res['answer'])



----------------------------------------------------------------------------------------------------
The conclusion of the paper is that artificial intelligence has a profound impact on legal applications, and the authors call for open-source legal language models to address the limitations of general-purpose language models in high-stakes decision-making. They propose creating a legal language model, called OpenJustice, and introduce the LegalQA dataset, highlighting the limitations of automatic evaluation processes when used on legal datasets.

----------------------------------------------------------------------------------------------------


In [97]:
for d in res['source_documents']:
    print("*"*50)
    print(d.metadata)
    print(d.page_content)

**************************************************
{'source': 'Dataset/Evaluating AI for Law.pdf', 'file_path': 'Dataset/Evaluating AI for Law.pdf', 'page': 0, 'total_pages': 16, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'pdfTeX-1.40.25', 'creationDate': 'D:20240419005128Z', 'modDate': 'D:20240419005128Z', 'trapped': ''}
Evaluating AI for Law: Bridging the Gap with
Open-Source Solutions
Rohan Bhambhoria1,2[0000−0002−2597−670X],
Samuel Dahan1,2,3[0000−0002−1079−8998],
Jonathan Li2[0000−0002−7095−805X], and
Xiaodan Zhu1,2[0000−0003−3856−3696]
1 Queen’s University, Kingston ON K7L3N6, Canada
{r.bhambhoria, samuel.dahan, jxl, xiaodan.zhu}@queensu.ca
2 Ingenuity Labs
3 Cornell University
Abstract. This study evaluates the performance of general-purpose AI,
like ChatGPT, in legal question-answering tasks, highlighting significant
risks to legal professionals and clients. It suggests leveraging founda-
tional m

In [98]:

res = chain("Do you know about the paper titled 'From Language Models to Practical Self-Improving Computer Agents'?")
print_readable(res['answer'])



----------------------------------------------------------------------------------------------------
The paper "From Language Models to Practical Self-Improving Computer Agents" proposes a methodology to create AI computer agents that can carry out diverse computer tasks and self-improve by developing tools and augmentations to enable themselves to solve increasingly complex tasks. The authors suggest that instead of manually developing static software to augment large language models (LLMs) through human engineering effort, an LLM agent can systematically generate software to augment itself. They demonstrate this approach through case studies, showing that an LLM agent can generate and use various augmentations, such as retrieval, internet search, web navigation, and text editor capabilities, to solve real-world computer tasks.

----------------------------------------------------------------------------------------------------


In [99]:

res = chain("What is the conclusion of this paper?")
print_readable(res['answer'])



----------------------------------------------------------------------------------------------------
The paper concludes that the proposed methodology enables an LLM agent to complete diverse tasks within a real-world computer system, and that the approach is both flexible due to its free-form augmentation capabilities and simple in algorithmic design. The authors also highlight the importance of effective prompt engineering in achieving intended outcomes. Additionally, they acknowledge the limitations of their existing implementation and invite future works to address these limitations, expecting that the effectiveness of their methodology will increase as more capable LLMs become available.

----------------------------------------------------------------------------------------------------


In [100]:
for d in res['source_documents']:
    print("*"*50)
    print(d.metadata)
    print(d.page_content)

**************************************************
{'source': 'Dataset/Practical Self-Improving Computer Agents.pdf', 'file_path': 'Dataset/Practical Self-Improving Computer Agents.pdf', 'page': 0, 'total_pages': 25, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'GPL Ghostscript 10.01.2', 'creationDate': "D:20240418202537-04'00'", 'modDate': "D:20240418202537-04'00'", 'trapped': ''}
arXiv:2404.11964v1  [cs.AI]  18 Apr 2024
From Language Models to Practical Self-Improving
Computer Agents
Alex Sheng
New York University
alexsheng4@gmail.com
Abstract
We develop a simple and straightforward methodology to create AI computer
agents that can carry out diverse computer tasks and self-improve by develop-
ing tools and augmentations to enable themselves to solve increasingly complex
tasks. As large language models (LLMs) have been shown to beneﬁt from non-
parametric augmentations, a signiﬁcant body of recent work has